# Coupling Coefficients for Pairwise Power Spectrum Estimators
This notebook computes the relevant coupling coefficients for the pairwise power spectrum and 2PCF estimators using both the midpoint and bisector line-of-sight definitions. These are used in the ```Beyond Yamamoto Estimators``` notebook.

**Dependencies:**
- python2 or python3
- scipy
- sympy

In [1]:
%pylab inline
from scipy.special import binom,factorial,factorial2
from sympy.physics.wigner import wigner_3j
from scipy.special import legendre
from scipy.special import gamma as gamma_fn

Populating the interactive namespace from numpy and matplotlib


### General Definitions

In [2]:
# Maximum alpha (midpoint)
AMAX = 4
# Maximum k (bisector)
KMAX = 2

def ACoeff(ell, m, lam, mu):
    return np.sqrt((4.*np.pi)*(2.*ell+1.)/(2.*lam+1.)/(2.*ell-2*lam+1.))*(binom(ell+m,lam+mu)*binom(ell-m,lam-mu))**(0.5)
def GegenCoeff(n,k,alpha):
    return (-1.)**k*gamma_fn(alpha+n-k)/(gamma_fn(alpha)*factorial(k)*factorial(n-2.*k))
def LegCoeff(L,n):
    return (2.*L+1.)*factorial(n)*factorial2(L+n+2)/(factorial(L+n+2)*factorial2(n-L))
def GauntCoeff(L1,L2,L3,M1,M2,M3):
    pref = np.sqrt((2*L1+1.)*(2.*L2+1.)*(2.*L3+1.)/(4.*np.pi))
    return pref*float(wigner_3j(L1,L2,L3,M1,M2,M3)*wigner_3j(L1,L2,L3,0,0,0))

### 1. Define Midpoint Legendre Coefficients
This computes the $f^{\alpha,\ell}_J$, $h^{\alpha,\ell\alpha_0}_J$ and $F^{\alpha,\ell}_J$ coefficients required for the Legendre shift theorem, the generalized Legendre shift theorem and the even-parity Legendre shift theorem.

In [3]:
def fCoeff(alpha,ell,J):
    if ell==0:
        return (alpha==0)*(J==0)
    out = 0.
    t1 = (2.*ell+1.)**(3./2.)*(-1.)**ell
    for lam in range(0,min(alpha,ell)+1):
        t2 = binom(2*ell,2*lam)**(0.5)*float(wigner_3j(ell,lam,ell-lam,0,0,0))
        if t2==0: continue
        for k in range(0,int(floor((alpha-lam)/2))+1):
            t3 = GegenCoeff(alpha-lam,k,ell/2.)*(-2.)**(alpha-lam-2*k)
            if t3==0: continue
            for L in range(alpha-lam-2*k,-1,-2):
                t4 = LegCoeff(L,alpha-lam-2*k)*float(wigner_3j(ell-lam,L,J,0,0,0))**2.
                if t4==0: continue
                out += t1*t2*t3*t4
    return out


def hCoeff(alpha,alpha0,ell,J):
    out = 0.
    t1 = (2.*ell+1.)**(3./2.)*(-1.)**ell
    for lam in range(0,min(alpha,ell)+1):
        t2 = binom(2*ell,2*lam)**(0.5)*float(wigner_3j(ell,lam,ell-lam,0,0,0))
        if t2==0: continue
        for k in range(0,int(floor((alpha-lam)/2))+1):
            t3 = GegenCoeff(alpha-lam,k,(ell+alpha0)/2.)*(-2.)**(alpha-lam-2*k)
            if t3==0: continue
            for L in range(alpha-lam-2*k,-1,-2):
                t4 = LegCoeff(L,alpha-lam-2*k)*float(wigner_3j(ell-lam,L,J,0,0,0))**2.
                if t4==0: continue
                out += t1*t2*t3*t4
    return out

def FCoeff(alpha,ell,J):
    # r1->r2 method
    if ell==0:
        return (alpha==0)*(J==0)
    
    # only compute if even!
    if (-1.)**alpha==-1: return 0.
    
    out = fCoeff(alpha,ell,J)
    
    if alpha>1:
        for beta in range(1,alpha+1,2): # odd beta
            for S in range(1,ell+beta+1,2): # odd S
                out += -0.5*fCoeff(beta,ell,S)*2.**(alpha-beta)*hCoeff(alpha-beta,beta,S,J)
           
    if alpha>3:
        for beta in range(1,alpha+1,2): # odd beta
            for S in range(1,ell+beta+1,2): # odd S
                f1 = fCoeff(beta,ell,S)*2.**(alpha-beta)
                if f1==0: continue
                for gamma in range(2,alpha-beta+1,2): # even gamma
                    for T in range(1,S+gamma+1,2): # odd T
                        f2 = hCoeff(gamma,beta,S,T)*hCoeff(alpha-beta-gamma,beta+gamma,T,J)
                        if f2==0: continue
                        out += 0.25*f1*f2
    if alpha>5: 
        raise Exception("not yet implemented")
    return out

def FCoeff2(alpha,ell,J):
    # alternative method
    if ell==0:
        return (alpha==0)*(J==0)
    
    out = 0.
    
    if alpha==0: 
        out = 1.*(J==ell)
    
    if alpha>1:
        out += -fCoeff(alpha,ell,J)
        
    if alpha>3:
        for beta in range(2,alpha+1,2):
            for S in range(0,ell+beta+1,2):
                f1 = fCoeff(beta,ell,S)
                if f1==0: continue
                for gamma in range(2,alpha-beta+1,2):
                    if (beta+gamma)!=alpha: continue
                    out += f1*hCoeff(gamma,beta,S,J)
    if alpha>5:
        for beta in range(2,alpha+1,2):
            for S in range(0,ell+beta+1,2):
                f1 = fCoeff(beta,ell,S)
                if f1==0: continue
                for gamma in range(2,alpha-beta+1,2):
                    for T in range(0,gamma+S+1,2):
                        f2 = hCoeff(gamma,beta,S,T)
                        if f2==0: continue
                        for delta in range(2,alpha-beta-gamma+1,2):
                            if (beta+gamma+delta)!=alpha: continue
                            out += -f1*f2*hCoeff(delta,beta+gamma,T,J)

    if alpha>7: raise Exception("not yet implemented")
    
    return out

In [4]:
# Generate All Parity Coefficients

leg_coeffs_all = []
for ell in [0,2,4]:
    for alpha in range(AMAX+1):
        for L in range(0,ell+alpha+1):
            weight = fCoeff(alpha,ell,L)
            if abs(weight)<1e-8: continue
            leg_coeffs_all.append([ell,alpha,L,weight])
leg_coeffs_all = np.asarray(leg_coeffs_all)
np.savetxt('/projects/QUIJOTE/Oliver/beyond_yama/leg_coeffs_midpoint_all.txt',leg_coeffs_all)

In [5]:
# Generate Even Parity Coefficiens

leg_coeffs_even = []
for ell in [0,2,4]:
    for alpha in range(0,AMAX+1,2):
        for L in range(0,ell+alpha+1,2):
            if ell==0:
                if alpha==0: weight = 1
                else: weight = 0.
            else:
                weight = FCoeff2(alpha,ell,L)
            if abs(weight)<1e-8: continue
            leg_coeffs_even.append([ell,alpha,L,weight])
leg_coeffs_even = np.asarray(leg_coeffs_even)
np.savetxt('/projects/QUIJOTE/Oliver/beyond_yama/leg_coeffs_midpoint_even.txt',leg_coeffs_even)

### 2. Define Midpoint Spherical Harmonic Coefficients
This computes the $\phi^{\alpha,\ell m}_{J_1J_2M}$, $\omega^{\alpha,\ell m \alpha_0}_{J_1J_2M}$ or $\Omega^{\beta,\alpha J_1J_2mM}_{S_1S_2T}$ and $\Phi^{\alpha,\ell m}_{J_1J_2M}$ coefficients required for the spherical harmonic shift theorem, the generalized spherical harmonic shift theorem and the even-parity spherical harmonic shift theorem.

In [6]:
def phi(alpha,ell,m,J1,J2,M):
    out = 0.
    if ell==0:
        return np.sqrt(4.*np.pi)*(J1==0)*(M==0)*(J2==0)
    
    for lam in range(0,ell+1):
        for mu in range(-lam,lam+1):
            tmpA = ACoeff(ell,m,lam,mu)
            for N in range(0,alpha+1):
                if alpha!=(lam+N):
                    continue
                for k in range(0,int(floor(N/2.))+1):
                    tmpC = GegenCoeff(N,k,ell/2.)*(-2.)**(N-2.*k)
                    for L in range(N-2*k,-1,-2):
                        tmpG = GauntCoeff(lam,L,J1,mu,M-mu,-M)*GauntCoeff(ell-lam,L,J2,m-mu,mu-M,M-m)
                        tmpL = LegCoeff(L,N-2*k)*4.*np.pi/(2.*L+1.)
                        tmp = tmpA*tmpC*tmpL*tmpG*(-1.)**(m+M-mu)
                        out += tmp
    return out

def omega(alpha,ell,m,alpha0,J1,J2,M):
    out = 0.
    for lam in range(0,ell+1):
        for mu in range(-lam,lam+1):
            tmpA = ACoeff(ell,m,lam,mu)
            for N in range(0,alpha+1):
                if alpha!=(lam+N):
                    continue
                for k in range(0,int(floor(N/2.))+1):
                    tmpC = GegenCoeff(N,k,(ell+alpha0)/2.)*(-2.)**(N-2.*k)
                    for L in range(N-2*k,-1,-2):
                        tmpG = GauntCoeff(lam,L,J1,mu,M-mu,-M)*GauntCoeff(ell-lam,L,J2,m-mu,mu-M,M-m)
                        tmpL = LegCoeff(L,N-2*k)*4.*np.pi/(2.*L+1.)
                        tmp = tmpA*tmpC*tmpL*tmpG*(-1.)**(m+M-mu)
                        out += tmp
    return out

def Omega(beta,alpha,J1,J2,m,M,S1,S2,T):
    out = 0.
    if(abs(m-M)>J2): return 0.
    for Jp in range(abs(J1-S1),J1+S1+1):
        tmpG = GauntCoeff(J1,Jp,S1,M,T-M,-T)
        if tmpG==0: continue
        tmp = omega(beta,J2,m-M,alpha,Jp,S2,T-M)*tmpG*(-1.)**T
        out += tmp
    return out

def Phi(alpha,ell,m,J1,J2,M):
    """r1->r2 method"""
    if ell==0:
        return np.sqrt(4.*np.pi)*(J1==0)*(M==0)*(J2==0)
    
    # catch odd coefficients!
    if (-1.)**alpha==-1:
        return 0. 
    
    # LO contribution
    out = phi(alpha,ell,m,J1,J2,M)
    if alpha<2: return out
    
    # NLO contribution
    out2 = 0.
    for beta in range(1,alpha+1,2):
        for J1p in range(1,beta+1,2):
            for J2p in range(max(1,ell-beta),ell+beta+1,2):
                for Mp in range(-J1p,J1p+1):
                    tmp_phi = phi(beta,ell,m,J1p,J2p,Mp)
                    if(tmp_phi==0): continue
                    tmp_Omega = Omega(alpha-beta,beta,J1p,J2p,m,Mp,J1,J2,M)
                    if(tmp_Omega==0): continue
                    tmp = 2.**(alpha-beta)*tmp_phi*tmp_Omega
                    out2 += tmp
    if alpha<4: return out-0.5*out2
    
    # NNLO contribution
    out4 = 0.
    for beta in range(1,alpha+1,2): # odd 
        for S1 in range(1,beta+1,2): # odd 
            for S2 in range(max(ell-beta,1),ell+beta+1,2): # odd
                for T in range(-S1,S1+1,1):
                    tmp_phi = phi(beta,ell,m,S1,S2,T)
                    if tmp_phi==0: continue
                    
                    for gamma in range(2,alpha-beta+1,2): # even
                        for Q1 in range(0,gamma+1,2): # even
                            for Q2 in range(max(S2-gamma,1),S2+gamma+1,2): # odd 
                                for R in range(-Q1,Q1+1,1):
                                    tmp_phi2 = omega(gamma,S2,m-T,beta,Q1,Q2,R)*2.**(alpha-beta)
                                    if tmp_phi2==0: continue

                                    for Q in range(abs(S1-Q1),S1+Q1+1,2): # from 3j
                                        g1 = GauntCoeff(S1,Q1,Q,T,R,-R-T)*(-1.)**(M-T-R)
                                        for X1 in range(abs(J1-Q),J1+Q+1,2): # from 3j
                                            g2 = GauntCoeff(Q,X1,J1,R+T,M-R-T,-M)
                                            tmp_phi3 = omega(alpha-beta-gamma,Q2,m-T-R,beta+gamma,X1,J2,M-T-R)
                                            if tmp_phi3==0: continue

                                            out4+=tmp_phi*tmp_phi2*tmp_phi3*g1*g2

    if alpha<6: return out-0.5*out2+0.25*out4
    
    raise Exception("alpha>4 not implemented yet!")
     
def Phi2(alpha,ell,m,J1,J2,M):
    """Alternative inversion method with Omega functions"""
    if alpha==0:
        return np.sqrt(4.*np.pi)*(ell==J2)*(J1==0.)*(M==0.)
    elif alpha==2:
        return -phi(2,ell,m,J1,J2,M) 
    elif alpha==4:
        out = -phi(4,ell,m,J1,J2,M) 
        for S1 in range(0,3,2): # even, beta = 2
            for S2 in range(max(ell-2,0),ell+3,2): # even
                for T in range(-S1,S1+1,1):
                    tmp_phi = phi(2,ell,m,S1,S2,T)
                    if tmp_phi==0: continue
                    tmp_Omega = Omega(2,2,S1,S2,m,T,J1,J2,M)
                    if tmp_Omega==0: continue
                    out += tmp_phi*tmp_Omega
        return out
    elif alpha==6:
        # first term
        out = -phi(6,ell,m,J1,J2,M)
        
        # second term
        for beta in [2,4]:
            gamma = alpha-beta
            for S1 in range(0,beta+1,2): # even
                for S2 in range(max(ell-beta,0),ell+beta+1,2): # even
                    for T in range(-S1,S1+1,1):
                        tmp_phi = phi(beta,ell,m,S1,S2,T)
                        if tmp_phi==0: continue
                        tmp_Omega = Omega(gamma,beta,S1,S2,m,T,J1,J2,M)
                        if tmp_Omega==0: continue
                        out += tmp_phi*tmp_Omega
                        
        # third term
        for S1 in range(0,3,2): # even, beta = 2
            for S2 in range(max(ell-2,0),ell+3,2): # even
                for T in range(-S1,S1+1,1):
                    tmp_phi = phi(2,ell,m,S1,S2,T)
                    if tmp_phi==0: continue
                    for Q1 in range(max(S1-2,0),S1+3,2): # even, gamma = 2
                        for Q2 in range(max(S2-2,0),S2+3,2): # even
                            for R in range(-Q1,Q1+1,1):
                                tmp_Omega = Omega(2,2,S1,S2,m,T,Q1,Q2,R)
                                if tmp_Omega==0: continue
                                tmp_Omega2 = Omega(2,4,Q1,Q2,m,R,J1,J2,M)
                                out += -tmp_phi*tmp_Omega*tmp_Omega2
        return out
    else:
        raise Exception("alpha>6 not implemented yet!")

In [7]:
# Generate All Parity Coefficients

ylm_coeffs_all = []
for ell in [0,2,4]:
    for m in range(-ell,ell+1,1):
        print(ell,m)
        for alpha in range(AMAX+1):
            for J1 in range(0,alpha+1):
                for J2 in range(max(ell-alpha,0),ell+alpha+1):
                    for M in range(-J1,J1+1,1):
                        if abs(m-M)>J2: continue
                        weight = phi(alpha,ell,m,J1,J2,M)
                        if abs(weight)<1e-8: continue
                        ylm_coeffs_all.append([ell,m,alpha,J1,J2,M,weight])
ylm_coeffs_all = np.asarray(ylm_coeffs_all)
np.savetxt('/projects/QUIJOTE/Oliver/beyond_yama/ylm_coeffs_midpoint_all.txt',ylm_coeffs_all)

(0, 0)
(2, -2)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(4, -4)
(4, -3)
(4, -2)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


In [8]:
# Generate Even Parity Coefficients

ylm_coeffs_even = []
for ell in [0,2,4]:
    for m in range(-ell,ell+1,1):
        print(ell,m)
        for alpha in range(0,AMAX+1,2):
            for J1 in range(0,alpha+1,2):
                for J2 in range(max(ell-alpha,0),ell+alpha+1,2):
                    for M in range(-J1,J1+1,1):
                        if abs(m-M)>J2: continue
                        weight = Phi2(alpha,ell,m,J1,J2,M)
                        if abs(weight)<1e-8: continue
                        ylm_coeffs_even.append([ell,m,alpha,J1,J2,M,weight])
ylm_coeffs_even = np.asarray(ylm_coeffs_even)
np.savetxt('/projects/QUIJOTE/Oliver/beyond_yama/ylm_coeffs_midpoint_even.txt',ylm_coeffs_even)

(0, 0)
(2, -2)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(4, -4)
(4, -3)
(4, -2)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)


### 3. Define Bisector Spherical Harmonic Coefficients
This computes the $B^{k,\ell m}_{J_1J_2S}$ coefficients required for the bisector spherical harmonic expansions.

In [9]:
def binom_general(x,y):
    # generalized binomial coefficient
    assert x<=0
    out = 1
    for i in range(y):
        out*=(x-i)
    return out/factorial(y)

In [10]:
def B_coeff(J1,J2,S,ell,m,k):
    out = 0.
    tmp1 = 2.**(-ell)
    for lam in range(0,ell+1):
        for mu in range(-lam,lam+1):
            tmp2 = ACoeff(ell,m,lam,mu)*binom_general(-ell/2.,k)*2.**(-k)
            for n in range(k+1):
                tmp3 = binom(k,n)*(-1.)**(k-n)
                for L in range(n,-1,-2):
                    tmp4 = LegCoeff(L,n)*4.*np.pi/(2.*L+1.)*(-1.)**(S-mu-m)*GauntCoeff(lam,L,J1,mu,S-mu,-S)*GauntCoeff(ell-lam,L,J2,m-mu,mu-S,S-m)
                    out += tmp1*tmp2*tmp3*tmp4
    return out

In [11]:
# Output bisector coefficients
ylm_coeffs_bis = []
for ell in [0,2,4]:
    for m in range(-ell,ell+1,1):
        print(ell,m)
        for k in range(KMAX+1):
            for J1 in range(0,ell+k+1):
                for J2 in range(0,ell+k+1):
                    for S in range(-J1,J1+1,1):
                        tmpB = B_coeff(J1,J2,S,ell,m,k)
                        if abs(tmpB)<1e-8: continue
                        ylm_coeffs_bis.append([ell,m,k,J1,J2,S,tmpB])
ylm_coeffs_bis = np.asarray(ylm_coeffs_bis)
np.savetxt('/projects/QUIJOTE/Oliver/beyond_yama/ylm_coeffs_bisector.txt',ylm_coeffs_bis)

(0, 0)
(2, -2)
(2, -1)
(2, 0)
(2, 1)
(2, 2)
(4, -4)
(4, -3)
(4, -2)
(4, -1)
(4, 0)
(4, 1)
(4, 2)
(4, 3)
(4, 4)
